In [1]:
%config Completer.use_jedi=False

In [2]:
from fastai.vision.all import *

In [3]:
images_path = Path('/nfs_storage/fs-mnt6/vaibhavg/data/images/male/')
assert images_path.exists()

In [4]:
images_list = get_image_files(images_path, recurse=False)

In [5]:
def get_x(elem):
    return Path(elem)

def get_y(row):
    return [0.0 for _ in range(15)]

In [6]:
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x = get_x,
    get_y = get_y,
    splitter=RandomSplitter(),
    item_tfms=(Resize(224)),
    batch_tfms=[
        FlipItem(p=0.5),
        Brightness(max_lighting=0.3, p=0.7, draw=None, batch=False),
        Saturation(max_lighting=0.3, p=0.7, draw=None, batch=False),
        Hue(max_hue=0.1, p=0.75, draw=None, batch=False),
        RandomErasing(p=0.2, sl=0.0, sh=0.15, max_count=6, min_aspect=0.2)
    ],
)

In [7]:
dls = dblock.dataloaders(images_list)

In [8]:
learner = cnn_learner(dls, resnet18)

In [9]:
learner.predict(images_list[0])

((0.2641298770904541,
  -1.9832581281661987,
  1.3707842826843262,
  0.9456096291542053,
  -5.0808563232421875,
  -4.248899936676025,
  5.36594820022583,
  4.451999664306641,
  -0.2664620876312256,
  -1.0456044673919678,
  -1.9824739694595337,
  -2.1637892723083496,
  -1.431849718093872,
  3.440964937210083,
  2.5792007446289062),
 tensor([ 0.2641, -1.9833,  1.3708,  0.9456, -5.0809, -4.2489,  5.3659,  4.4520,
         -0.2665, -1.0456, -1.9825, -2.1638, -1.4318,  3.4410,  2.5792]),
 tensor([ 0.2641, -1.9833,  1.3708,  0.9456, -5.0809, -4.2489,  5.3659,  4.4520,
         -0.2665, -1.0456, -1.9825, -2.1638, -1.4318,  3.4410,  2.5792]))

In [5]:
learner_path = Path('trp_wt=grp_wt=1_non_progressive_full_face.pkl')

def get_x(row):
    return Path(row['human_img_path'])

def get_y(row):
    return get_lips_params_avatar_imagename(row['winner']), \
           get_lips_params_avatar_imagename(row['loser']), \
           row['round_num']

def loss(pred, label, trp_wt=1, group_loss_wt=1, point_loss_wt=0, MARGIN_TRIPLET=0.2):

    def triplet_loss(pred, label, MARGIN=MARGIN_TRIPLET):
        ''' Analysis can be seen in the notebook - loss_function '''
        winner, loser, round_num = label

        winner_dist = ((pred - winner) ** 2).mean(axis=1)
        loser_dist = ((pred - loser) ** 2).mean(axis=1)

        loss = winner_dist - loser_dist + MARGIN
        return torch.relu(loss).mean()

    def group_loss(pred, label):
        winner, loser, round_num = label
        return (nn.MSELoss()(pred, winner) + nn.MSELoss()(pred, loser)) / 2

    def point_loss(pred, label, thresh = 2.9):
        winner, loser, round_num = label
        bs = winner.shape[0]

        num_rounds_thresh = torch.where(round_num >= thresh, 1, 0).sum()
        pt_loss_sum = torch.where(round_num > thresh, 
                          ((pred - winner) ** 2).mean(axis=1), 
                          torch.zeros((bs, )).float().to(device)).sum()

        return torch.where( num_rounds_thresh > 0, pt_loss_sum / num_rounds_thresh, torch.tensor(0).float().to(device) )


    return trp_wt * triplet_loss(pred, label) + \
           group_loss_wt * group_loss(pred, label) + \
           point_loss_wt * point_loss(pred, label)

def splitter(df):
    ''' We need to ensure that different images be used for training and validation, and not merely different
        triplets for same images '''
    TRAIN_SIZE = 0.8
    VALID_SIZE = 1.0 - TRAIN_SIZE
    train_idx = list( range(int(TRAIN_SIZE * len(df))) )
    valid_idx = list( range(int(TRAIN_SIZE * len(df)), len(df)) )
    return train_idx, valid_idx

def mae_winner(pred, label):
    winner, loser, round_num = label
    return nn.L1Loss()(pred, winner)

def mae_loser(pred, label):
    winner, loser, round_num = label
    return nn.L1Loss()(pred, loser)


assert learner_path.exists()
learner = load_learner(learner_path)

In [11]:
learner.dls = dls

In [15]:
preds = learner.predict(images_list[0])
preds

((-0.29914921522140503,
  -0.07510409504175186,
  -0.38681042194366455,
  0.10719355940818787,
  0.022745639085769653,
  0.671205997467041,
  0.14725103974342346,
  0.10758504271507263,
  0.19371020793914795,
  -0.4709028899669647,
  -0.23201623558998108,
  -0.04340946674346924,
  -0.10809613019227982,
  0.4655865430831909,
  0.09255486726760864),
 tensor([-0.2991, -0.0751, -0.3868,  0.1072,  0.0227,  0.6712,  0.1473,  0.1076,
          0.1937, -0.4709, -0.2320, -0.0434, -0.1081,  0.4656,  0.0926]),
 tensor([-0.2991, -0.0751, -0.3868,  0.1072,  0.0227,  0.6712,  0.1473,  0.1076,
          0.1937, -0.4709, -0.2320, -0.0434, -0.1081,  0.4656,  0.0926]))

In [13]:
def unnormalize_learner_preds(preds):
    mean = np.array([-2.285041, 19.212197, -16.371056, -2.285041, 19.212197, 16.371056, 1.150694, -1.504351, 
                     -4.001261, 0.323188, -19.492486, 0.000000, 0.000000, 9.950089, -19.391928])
    std = np.array([7.615347, 8.641896, 17.199984, 7.615347, 8.641896, 17.199984, 9.583560, 5.826328, 
                    10.115759, 1.251700, 10.740775, 1.000000, 1.000000, 13.113880, 39.773768])
    return np.array(preds) * std + mean

In [16]:
unnormalize_learner_preds(preds[0])

array([ -4.56316608,  18.56315522, -23.02418907,  -1.46872485,
        19.40876245,  27.91578842,   2.56188317,  -0.87752525,
        -2.04173522,  -0.26624115, -21.98452018,  -0.04340947,
        -0.10809613,  16.05573506, -15.71067218])

PicklingError: Can't pickle <function get_x at 0x7fa40f0fc5e0>: it's not the same object as __main__.get_x